# Image corpus creation and Transfer Learning in CNN

### Suriya S (225229140)

**Steps**

### 1. Import the necessary libraries

In [65]:
import datetime
import keras
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

### 2. Initialize some parameters

In [20]:
now = datetime.datetime.now  #get current time

batch_size = 128
num_classes = 5
epochs = 5

img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

### 3.Partition MINST dataset

In [5]:
#data, shuffled and split between train and test sets

(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [24]:
#create 2 datasets: one with digits below 5 and one with 5 and above

x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [25]:
x_train_lt5.shape

(30596, 28, 28)

In [26]:
x_train_gte5.shape

(29404, 28, 28)

### 4. Define the "feature" layers

In [55]:
feature_layers =[ 
    Conv2D(filters, kernel_size = (3,3), activation = 'relu',padding='same',input_shape = (img_rows,img_cols, 1)),
    Conv2D(filters, kernel_size = (3,3), activation = 'relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Flatten() 
]

### 5. Define the "classification" layers

In [56]:
classification_layers = [
    Dense(128, activation = 'relu'),
    Dropout(0.5),
    Dense(num_classes, activation = 'softmax')
]

### 6. Define a Sequential model

In [57]:
model = Sequential(feature_layers+classification_layers)
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 conv2d_16 (Conv2D)          (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 dropout_16 (Dropout)        (None, 13, 13, 32)        0         
                                                                 
 flatten_7 (Flatten)         (None, 5408)              0         
                                                                 
 dense_17 (Dense)            (None, 128)               692352    
                                                      

### 7. Create function train_model()

In [66]:
def train_model(model,train,test,num_classes):
    train[0] = train[0].reshape(train[0].shape[0],img_rows,img_cols,1)
    test[0] = test[0].reshape(test[0].shape[0],img_rows,img_cols,1)

    train[0] = train[0].astype('float32')
    test[0] = test[0].astype('float32')

    train[0] /= 255
    test[0] /= 255

    print(train[0].shape)
    print(test[0].shape)
 
    s = pd.Series(train[1])
    train[1] = pd.get_dummies(s)
    train[1] = train[1].values

    s = pd.Series(test[1])
    test[1] = pd.get_dummies(s)
    test[1] = test[1].values

    model.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])

    train_start_time=now()

    model.fit(train[0],train[1],epochs=epochs,verbose=2)
  
    print('\n')
    print('\n')

    print('Training time: %s' % (now() - train_start_time))

    score=model.evaluate(test[0],test[1],verbose=0)

    print('test loss ',score[0])
    print('test accuracy ',score[1])

### 8. Training on digits 5 to 9

In [67]:
train_model(model, [x_train_gte5, y_train_gte5], [x_test_gte5, y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 58s - loss: 1.5841 - accuracy: 0.2836 - 58s/epoch - 63ms/step
Epoch 2/5
919/919 - 56s - loss: 1.5033 - accuracy: 0.4696 - 56s/epoch - 61ms/step
Epoch 3/5
919/919 - 56s - loss: 1.4056 - accuracy: 0.5872 - 56s/epoch - 60ms/step
Epoch 4/5
919/919 - 57s - loss: 1.2817 - accuracy: 0.6525 - 57s/epoch - 63ms/step
Epoch 5/5
919/919 - 56s - loss: 1.1352 - accuracy: 0.7077 - 56s/epoch - 61ms/step




Training time: 0:04:43.643675
test loss  0.997369647026062
test accuracy  0.843036413192749


### 9. Freeze Feature Layers

In [68]:
#Freeze only the feature layers

for l in feature_layers:
  l.trainable = False

### 10. Print Summary

In [71]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 conv2d_16 (Conv2D)          (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 dropout_16 (Dropout)        (None, 13, 13, 32)        0         
                                                                 
 flatten_7 (Flatten)         (None, 5408)              0         
                                                                 
 dense_17 (Dense)            (None, 128)               692352    
                                                      

### 11. Training for digits 0 to 4 based on digits 5 to 9

In [72]:
train_model(model, [x_train_lt5, y_train_1t5], [x_test_1t5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 19s - loss: 1.4384 - accuracy: 0.4137 - 19s/epoch - 20ms/step
Epoch 2/5
957/957 - 18s - loss: 1.2196 - accuracy: 0.6080 - 18s/epoch - 19ms/step
Epoch 3/5
957/957 - 18s - loss: 1.0473 - accuracy: 0.7366 - 18s/epoch - 19ms/step
Epoch 4/5
957/957 - 18s - loss: 0.9139 - accuracy: 0.7984 - 18s/epoch - 19ms/step
Epoch 5/5
957/957 - 20s - loss: 0.8066 - accuracy: 0.8298 - 20s/epoch - 21ms/step




Training time: 0:01:33.398016
test loss  0.6758227944374084
test accuracy  0.9073749780654907


### 12. Reversing the training process

In [73]:
modelReverse = Sequential(feature_layers+classification_layers)
modelReverse.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 conv2d_16 (Conv2D)          (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 dropout_16 (Dropout)        (None, 13, 13, 32)        0         
                                                                 
 flatten_7 (Flatten)         (None, 5408)              0         
                                                                 
 dense_17 (Dense)            (None, 128)               692352    
                                                      

In [75]:
train_model(modelReverse, [x_train_lt5, y_train_1t5], [x_test_1t5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 19s - loss: 0.7184 - accuracy: 0.8522 - 19s/epoch - 20ms/step
Epoch 2/5
957/957 - 22s - loss: 0.6465 - accuracy: 0.8680 - 22s/epoch - 23ms/step
Epoch 3/5
957/957 - 18s - loss: 0.5894 - accuracy: 0.8777 - 18s/epoch - 19ms/step
Epoch 4/5
957/957 - 19s - loss: 0.5424 - accuracy: 0.8850 - 19s/epoch - 20ms/step
Epoch 5/5
957/957 - 20s - loss: 0.5039 - accuracy: 0.8911 - 20s/epoch - 21ms/step




Training time: 0:01:38.502933
test loss  0.39636582136154175
test accuracy  0.9394823908805847


In [ ]:
#Freeze only the feature layers

for l in feature_layers:
  l.trainable = False

In [78]:
modelReverse.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 conv2d_16 (Conv2D)          (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 dropout_16 (Dropout)        (None, 13, 13, 32)        0         
                                                                 
 flatten_7 (Flatten)         (None, 5408)              0         
                                                                 
 dense_17 (Dense)            (None, 128)               692352    
                                                      

In [77]:
train_model(modelReverse, [x_train_gte5, y_train_gte5], [x_test_gte5, y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 18s - loss: 1.2907 - accuracy: 0.4943 - 18s/epoch - 20ms/step
Epoch 2/5
919/919 - 18s - loss: 1.1162 - accuracy: 0.5713 - 18s/epoch - 19ms/step
Epoch 3/5
919/919 - 19s - loss: 0.9863 - accuracy: 0.6439 - 19s/epoch - 21ms/step
Epoch 4/5
919/919 - 18s - loss: 0.8917 - accuracy: 0.6925 - 18s/epoch - 20ms/step
Epoch 5/5
919/919 - 18s - loss: 0.8212 - accuracy: 0.7285 - 18s/epoch - 19ms/step




Training time: 0:01:30.628136
test loss  0.6975718140602112
test accuracy  0.7988068461418152
